<a href="https://colab.research.google.com/github/nimishwork6699/Deep_Learning_Pytorch/blob/main/fullANN_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
# Set manual seed
torch.manual_seed(42)

In [10]:
# check gpu availiblity
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device :{device}")

Using device :cuda


In [11]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fashionmnist' dataset.
Path to dataset files: /kaggle/input/fashionmnist


In [12]:
df = pd.read_csv(path + "/fashion-mnist_train.csv")

In [13]:
# train test split
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
# scaling the features
X_train=X_train/255.0
X_test=X_test/255.0

In [16]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00392157, 0.        ,
        0.        ]])

In [17]:
class MyCoustomDataset(Dataset):

  def __init__(self,feature,labels):
    self.feature=torch.tensor(feature,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)

  def __len__(self):

    return len(self.feature)

  def __getitem__(self, index):
    return self.feature[index],self.labels[index]


In [18]:
# creating train_dataset
# features and labels ki jagah we are making it as a subsitute
train_dataset=MyCoustomDataset(X_train,y_train)

In [19]:
# creating test dataset
test_dataset=MyCoustomDataset(X_test,y_test)

In [21]:
# creating train and test loader
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [22]:
# define NN Module
class MyNN(nn.Module):
  def __init__(self, num_features):

    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,X):
    return self.model(X)

In [23]:
# set learning rate and epochs
epochs = 100
learning_rate = 0.1

In [25]:
# instatiate the model
model = MyNN(X_train.shape[1])
model=model.to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr= learning_rate)

In [30]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # moving data to gpu
    batch_features, batch_labels=batch_features.to(device),batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 0.6456927449802558
Epoch: 2 , Loss: 0.4343211544752121
Epoch: 3 , Loss: 0.3885515774885813
Epoch: 4 , Loss: 0.3623336294392745
Epoch: 5 , Loss: 0.3411979767108957
Epoch: 6 , Loss: 0.3248021237651507
Epoch: 7 , Loss: 0.3098141959955295
Epoch: 8 , Loss: 0.2996180685410897
Epoch: 9 , Loss: 0.28877133907129365
Epoch: 10 , Loss: 0.2767990078938504
Epoch: 11 , Loss: 0.2703479566586514
Epoch: 12 , Loss: 0.26266809437672295
Epoch: 13 , Loss: 0.2550013987707595
Epoch: 14 , Loss: 0.24656374812374512
Epoch: 15 , Loss: 0.2394655180660387
Epoch: 16 , Loss: 0.23421444064254562
Epoch: 17 , Loss: 0.22625318068576356
Epoch: 18 , Loss: 0.22329780512303113
Epoch: 19 , Loss: 0.2201658262523512
Epoch: 20 , Loss: 0.21307433107743662
Epoch: 21 , Loss: 0.2088468018732965
Epoch: 22 , Loss: 0.2025416794841488
Epoch: 23 , Loss: 0.20226349614498515
Epoch: 24 , Loss: 0.19529123815397423
Epoch: 25 , Loss: 0.188763703393129
Epoch: 26 , Loss: 0.1853718101118381
Epoch: 27 , Loss: 0.1825562600698322
Ep

In [31]:
# set model to eval mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [32]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:
     # moving data to gpu
    batch_features, batch_labels=batch_features.to(device),batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)


0.88625


In [33]:
len(test_loader)

375